In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

import h2o
import create_data as create
import pandas as pd
import utilities as utils

from h2o.automl import H2OAutoML

In [ ]:
data = create.CreateTrainingData()
train = data.create_training_data()
test = create.create_test_data()
target = create.create_target_data()

In [ ]:
h2o.init()

In [ ]:
train.head()

In [ ]:
# Fill nans with 0 in training data
train.fillna(0, inplace=True)

X_train = h2o.H2OFrame(train)
y = 'pv_measurement'

X = X_train.columns
X.remove(y)

In [ ]:
autoML = H2OAutoML(max_models=30, seed=1, nfolds=5)
autoML.train(x=X, y=y, training_frame=X_train)

In [11]:
# Remove features that are not in the training set
test.fillna(0, inplace=True)
test_pd = test[X]
test_h2o = h2o.H2OFrame(test_pd)

pred = autoML.predict(test_h2o)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


H2OValueError: No AutoML instance with id None.

In [ ]:
prediction_no_edit = pred.as_data_frame()
prediction_with_filter = pred.as_data_frame()

prediction_with_filter[prediction_with_filter < 0] = 0

# Add index to predictions in column left of predictions
prediction_no_edit.insert(0, 'id', range(0, len(prediction_no_edit)))
prediction_with_filter.insert(0, 'id', range(0, len(prediction_with_filter)))    

# Remove column 'date_forecast'
prediction_no_edit.drop(columns=['date_forecast'], inplace=True)
prediction_with_filter.drop(columns=['date_forecast'], inplace=True)

# Save predictions to csv in folder 'results'
prediction_no_edit.to_csv('results/prediction_basic_nans_to_0.csv', index=False)
prediction_with_filter.to_csv('results/prediction_with_filter_basic_nans_to_0.csv', index=False)